In [1]:
import os
import numpy as np
import pandas as pd

path = '/Users/doyoung/학교/Python_Programming_JupyterNotebook/데이터/iris.csv'
data = pd.read_csv(path)

In [2]:
Y = data['class']
X = data.drop(['class'], axis=1)

In [3]:
def train_test_split(X, Y, test_size=0.25,random=False, random_seed=None):
    X = np.array(X)
    Y = np.array(Y)
    
    indices = np.array(range(len(X)))
    test_size_len = round(test_size * X.shape[0])
    
    if random==True:
        if random_seed != None:
            random_generator = np.random.RandomState(seed=random_seed)
            random_generator.shuffle(indices)
            
        else:
            np.random.shuffle(indices)
            
        test_indices = indices[0:test_size_len]
        train_indices = indices[test_size_len:]
        
        X_train = X[train_indices, :]
        X_test = X[test_indices, :]
        Y_train = Y[train_indices]
        Y_test = Y[test_indices]
        
    else:
        train_indices = indices[0:(len(X)-test_size_len)]
        test_indices = indices[(len(X)-test_size_len)]
        
        X_train = X[train_indices, :]
        X_test = X[test_indices, :]
        Y_train = Y[train_indices]
        Y_test = Y[test_indices]
        
    return X_train, X_test, Y_train, Y_test
        

In [4]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,random=True,test_size=0.3,random_seed=50)

In [5]:
class knn:
    
    def __init__(self, k = 5):
        self.k = k
        self.neighbors = None
        
        
    def euclidean_dist(self, obs1, obs2):
        dist = np.sqrt(np.sum(obs1-obs2)**2)
        return np.around(dist,3)
    
    def search_neighbors(self, X_train, test_sample):
        dists_info=list()
        
        for index, train_sample in enumerate(X_train):
            dist = self.euclidean_dist(train_sample, test_sample)
            dists_info.append((train_sample,dist,index))
        dists_info.sort(key=lambda tupe:tupe[1])
        self.neighbors = dists_info[:self.k]
        return self.neighbors
    
    def predict(self, X_train,Y_train, test_sample):
        self.neighbors = self.search_neighbors(X_train, test_sample)
        
        neigh_index = list()
        for neigh in self.neighbors:
            neigh_index.append(neigh[2])
            
        neigh_y = Y_train[neigh_index]
        
        classes, counts = np.unique(neigh_y, return_counts = True)
        pred=classes[np.argmax(counts)]
        return pred,neigh_y

In [6]:
knn = knn(k=5)

In [7]:
total_pred = list()
for index, test_sample in enumerate(X_test):
    pred, neigh_y = knn.predict(X_train, Y_train, test_sample)
    total_pred.append(pred)

In [8]:
knn.neighbors

[(array([6.9, 3.2, 5.7, 2.3]), 0.0, 57),
 (array([7.3, 2.9, 6.3, 1.8]), 0.2, 25),
 (array([6.7, 3.3, 5.7, 2.1]), 0.3, 58),
 (array([6.3, 3.4, 5.6, 2.4]), 0.4, 52),
 (array([7.2, 3. , 5.8, 1.6]), 0.5, 43)]

In [9]:
total_accuracy = sum(total_pred==Y_test)/len(Y_test)
total_error_rate = 1 - total_accuracy

print("정확도 = {:.2f}".format((total_accuracy)*100))
print("오류율 = {:.2f}".format((total_error_rate)*100))

정확도 = 84.44
오류율 = 15.56
